
# **Importação do Drive com os arquivos**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



# **Importação inicial das bibliotecas**


####Fazer a importação das bibliotecas



In [ ]:
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
import numpy as np
import math
import re
import os
import pandas as pd
from bs4 import BeautifulSoup
import random
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
!pip install bert-for-tf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 952.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30509 sha256=f761d5e3003062fb330824c5f3e503ef4df01fedea3b3622be5bb68283b94fc1
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19454 sha256=18573c70b283f4cfd448f0f9ef64f8beace531d1dd7acca2189c74e6546de365
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=f9c58b2120bd419ec627b986836dcd445ae95497fd80d20d22b555fe133363fc
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e9087

In [ ]:
!pip install sentencepiece

In [ ]:
import tensorflow_hub as hub

In [ ]:
from tensorflow.keras import layers
import bert

In [ ]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

In [ ]:
def read_and_concatenate_dataframes(directory, new_column_name, value):
    """
    Read JSONL files in the specified directory, add a new column to each DataFrame,
    and concatenate all DataFrames into a single DataFrame.

    Parameters:
    - directory: The path to the directory containing JSONL files.
    - new_column_name: The name of the new column to be added.
    - value: The value to be assigned to the new column.

    Returns:
    A DataFrame containing all the concatenated data.
    """

    # List to store DataFrames
    data_frames_list = []

    # Iterate over files in the directory
    for file_name in os.listdir(directory):
        if file_name.endswith('.jsonl'):
            file_path = os.path.join(directory, file_name)

            # Read the JSONL file and add to the DataFrame list
            temp_df = pd.read_json(file_path, lines=True, encoding='latin1')

            # Add a new column with specified name and value
            temp_df[new_column_name] = value

            data_frames_list.append(temp_df)

    # Concatenate all DataFrames into a single DataFrame
    result_df = pd.concat(data_frames_list, ignore_index=True)

    return result_df

# Example usage
directory_pathG = '/content/drive/MyDrive/opengpttext-clean/chatgpt/'
directory_pathH = '/content/drive/MyDrive/opengpttext-clean/openweb/'
new_column = 'value'

# Call the function with value 1 for GPT data and value 0 for human data
data_gpt = read_and_concatenate_dataframes(directory_pathG, new_column, value=1)
data_human = read_and_concatenate_dataframes(directory_pathH, new_column, value=0)

In [ ]:
data_human

,uid,text,value
0,[urlsf_subset00]-[83],Check current weather conditions\nIt's going t...,0
1,[urlsf_subset00]-[89],"On Thursday, the president of the United State...",0
2,[urlsf_subset00]-[390],Curitiba -\nEl delantero Enner Valencia marco ...,0
3,[urlsf_subset00]-[457],Getting started\n(I'm using Windows 10)\nAfter...,0
4,[urlsf_subset00]-[458],"Today, Mexico's newest data retention law ente...",0
...,...,...,...
29137,[urlsf_subset09]-[389268],Academic freedom -- for conservatives -- died ...,0
29138,[urlsf_subset09]-[389288],When Dylan Higgins invited me on to the Field ...,0
29139,[urlsf_subset09]-[389386],John Kasich. AP Photo/John Minchillo\nJohn Kas...,0
29140,[urlsf_subset09]-[389484],A Navy-funded thermal engine bobbing off the c...,0


In [ ]:
data_gpt

,uid,text,value
0,[urlsf_subset01]-[362358],The US is to review its military support for S...,1
1,[urlsf_subset01]-[116340],Some game developers are seeking inspiration f...,1
2,[urlsf_subset01]-[299487],IntercontinentalExchange's acquisition of the ...,1
3,[urlsf_subset01]-[137018],According to research from The University of W...,1
4,[urlsf_subset01]-[302038],The Davao City Council has declared comedian R...,1
...,...,...,...
29390,[urlsf_subset05]-[141028],"The article ""Creative underexposure with a Nik...",1
29391,[urlsf_subset05]-[49964],The special education services that New York S...,1
29392,[urlsf_subset05]-[116459],"During the third week of the trial, two opposi...",1
29393,[urlsf_subset05]-[67867],"Mogwai Young Team, the debut studio album by S...",1


In [ ]:
data_human.shape

(29142, 3)

In [ ]:
data_gpt.shape

(29395, 3)

In [ ]:
# Concatenar os DataFrames em uma única variável
combined_data = pd.concat([data_gpt, data_human], ignore_index=True)
combined_data = combined_data.drop(combined_data.columns[0], axis=1)

In [ ]:
combined_data

,text,value
0,The US is to review its military support for S...,1
1,Some game developers are seeking inspiration f...,1
2,IntercontinentalExchange's acquisition of the ...,1
3,According to research from The University of W...,1
4,The Davao City Council has declared comedian R...,1
...,...,...
58532,Academic freedom -- for conservatives -- died ...,0
58533,When Dylan Higgins invited me on to the Field ...,0
58534,John Kasich. AP Photo/John Minchillo\nJohn Kas...,0
58535,A Navy-funded thermal engine bobbing off the c...,0


In [ ]:
combined_data.head()

,text,value
0,The US is to review its military support for S...,1
1,Some game developers are seeking inspiration f...,1
2,IntercontinentalExchange's acquisition of the ...,1
3,According to research from The University of W...,1
4,The Davao City Council has declared comedian R...,1


In [ ]:
combined_data.tail()

,text,value
58532,Academic freedom -- for conservatives -- died ...,0
58533,When Dylan Higgins invited me on to the Field ...,0
58534,John Kasich. AP Photo/John Minchillo\nJohn Kas...,0
58535,A Navy-funded thermal engine bobbing off the c...,0
58536,A U.S. Army Black Hawk helicopter is seen. (Ph...,0


# **- Limpeza dos Textos**


In [ ]:
def clean_text(text):
    # Remover menções a usuários
    text = re.sub(r"@[A-Za-z0-9_]+", '', text)
    # Remover URLs
    text = re.sub(r"https?://[A-Za-z0-9./]+", '', text)
    # Manter apenas letras, pontos de exclamação, interrogação e ponto final
    text = re.sub(r"[^A-Za-z.!?']", ' ', text)
    # Remover espaços extras
    text = re.sub(r" +", ' ', text)
    return text

In [ ]:
data_clean = [clean_text(tweet) for tweet in combined_data.text]

In [ ]:
data_clean[0:3]

["The US is to review its military support for Saudi Arabia as officials warn it will be influenced by whether the kingdom commits to a ceasefire with Houthi rebels in Yemen according to an anonymous source speaking at a press conference. The review follows a Saudi strike on a funeral in the Yemeni capital Sanaa on October which killed more than people. The attack stiffened US concerns about its role in the war which has led to the deaths of more than non combatants. Assistance may continue for the Saudi US border but this will depend on the review's internal deliberations.",
 "Some game developers are seeking inspiration for VR by looking to the past but Funktronix Labs has taken it one step further by releasing its first person VR RTS Cosmic Trip for the Virtual Boy. The game design studio was first taken seriously when it posted a mockup of its design on Reddit a few months ago. If we say we are going to target that platform you better freakin' believe that it is coming to that plat

In [ ]:
data_labels = combined_data.value.values
data_labels

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def encode_sentence(sent):
  return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
encode_sentence("I love before")

['[CLS]', 'i', 'love', 'before', '[SEP]']

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [ ]:
print(data_inputs[0:2])

[['[CLS]', 'the', 'us', 'is', 'to', 'review', 'its', 'military', 'support', 'for', 'saudi', 'arabia', 'as', 'officials', 'warn', 'it', 'will', 'be', 'influenced', 'by', 'whether', 'the', 'kingdom', 'commits', 'to', 'a', 'ceasefire', 'with', 'ho', '##uth', '##i', 'rebels', 'in', 'yemen', 'according', 'to', 'an', 'anonymous', 'source', 'speaking', 'at', 'a', 'press', 'conference', '.', 'the', 'review', 'follows', 'a', 'saudi', 'strike', 'on', 'a', 'funeral', 'in', 'the', 'yemen', '##i', 'capital', 'san', '##aa', 'on', 'october', 'which', 'killed', 'more', 'than', 'people', '.', 'the', 'attack', 'stiffened', 'us', 'concerns', 'about', 'its', 'role', 'in', 'the', 'war', 'which', 'has', 'led', 'to', 'the', 'deaths', 'of', 'more', 'than', 'non', 'combatants', '.', 'assistance', 'may', 'continue', 'for', 'the', 'saudi', 'us', 'border', 'but', 'this', 'will', 'depend', 'on', 'the', 'review', "'", 's', 'internal', 'del', '##ibe', '##rations', '.', '[SEP]'], ['[CLS]', 'some', 'game', 'developers

## Criação da Base de Dados

In [ ]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
np.char.not_equal("[PAD]", "[PAD]")

array(False)

In [ ]:
def get_mask(tokens):
  return np.char.not_equal(tokens, "[PAD]").astype(int)

In [ ]:
get_mask(tokenizer.tokenize("My dog likes strawberries."))

array([1, 1, 1, 1, 1, 1])

In [ ]:
def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
               get_mask(sent_lab[0]),
               get_segments(sent_lab[0])],
              sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [ ]:
sorted_all[0]

([[101,
   1996,
   20423,
   2003,
   2525,
   1999,
   3674,
   20423,
   2015,
   2061,
   2053,
   16360,
   13492,
   7741,
   4072,
   1012,
   102],
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 1)

In [ ]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [ ]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 6
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [ ]:
len(sorted_all)

58537

In [ ]:
NB_BATCHES_TEST

304